In [2]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import regex
import time

try:
    driver_path = r"C:\Program Files (x86)\ChromeDriver\chromedriver.exe"
    website = "https://www.jomashop.com/fragrances.html"
    
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1280,1440")
                            
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    driver.get(website)

    # Hitting ESC on the body element to close the modal that appears whenever you load the page.
    time.sleep(4)
    driver.find_element(by='xpath', value='//body').send_keys(Keys.ESCAPE)
    time.sleep(2)
    
    frags = driver.find_elements(by='xpath', value='//div[@class="product-details"]')
    
    def find_frags(driver):
        seen_frags = set()
        frag_brand = []
        frag_name = []
        frag_type = []
        #frag_gender = []
        frag_price = []
        frag_retail_price = []
        frag_discount = []
        page_counter = 0
        
        # Scroll the page incrementally 6 times and stop when the bottom is reached.
        def scroll_page(driver):
           
            scroll_count = 6
            last_height = driver.execute_script("return document.body.scrollHeight")
            scroll_increment = last_height // scroll_count
    
            for _ in range(scroll_count):
                driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
                time.sleep(1.5)
                new_height = driver.execute_script("return document.body.scrollHeight")
    
                if new_height == last_height:  
                    break
                last_height = new_height
        
        name_pattern = regex.compile(r"^(?:(Men's|Ladies|Unisex)\s+)?(?:-\s*)?([^/]+?)(?=\s?Spray\s?|\s*/|\s+by|\s+\(|\s+\d+\.\d+|\s+(?:Aftershave|Bath and Shower Products|Body Spray|Car Diffuser|Cleansers|Deodorant|Diffuser|Eau De Parfum|Eau De Cologne|Eau De Toilette|Extrait de Parfum|Eau de Cologne|Eau de Parfum|Eau de Toilette|Extrait De Parfum|Free Water|Gift Set|Lotion & Oils|Lotions & Creams|Mist|Oil & Serums|Perfume Oil|Room Fragrance|Room Spray|Scented Candle|Scented Cards|Scrubs, Foams & Exfoliants|Shower Gels|Soap|Solid Parfum|Tools|Wash|EDP|EDT|EDC|Cologne|Shower Gel|Shampoo / Shower Gel)|$)|\((m|u|w)\)", regex.IGNORECASE)
        type_pattern = regex.compile(r"(Aftershave|Bath and Shower Products|Body Spray|Car Diffuser|Cleansers|Deodorant|Diffuser|Eau De Parfum|Eau De Cologne|Eau De Toilette|Extrait de Parfum|Eau de Cologne|Eau de Parfum|Eau de Toilette|Extrait De Parfum|Free Water|Gift Set|Lotion & Oils|Lotions & Creams|Mist|Oil & Serums|Parfum|Perfume Oil|Room Fragrance|Room Spray|Scented Candle|Scented Cards|Scrubs, Foams & Exfoliants|Shower Gels|Soap|Solid Parfum|Tools|Wash|EDP|EDT|EDC|Cologne|Shower Gel|Shampoo / Shower Gel)", regex.IGNORECASE)
        
        # Set to 1 for testing, would technically be just any higher number than the total page count I guess (total results / 60 which is the # of items per page) since it stops when it reaches the last page anyway.
        # Use "while True" when finished testing as it will run until the last page is found (based on logic further down)
        
        # while True:
          while page_counter < 1:
            scroll_page(driver)

            new_frags = driver.find_elements(by='xpath', value='//div[contains(@class, "product-details")]')
            for frag in new_frags:
                if frag not in seen_frags:
                    # Avoids duplication by only adding unique fragrances to the set. Using a set as opposed to a list since we won't be pulling dupes and need this to not be slow. There will be like 250 pages lol.
                    seen_frags.add(frag)
                try:
                    orig_name = frag.find_element(by='xpath', value='.//span[@class="name-out-brand"]').text
                    name_match = regex.match(name_pattern, orig_name)
                    type_match = regex.search(type_pattern, orig_name)
                    
                    if name_match:
                        cleaned_name = name_match.group(2).strip()   
                    else:
                        cleaned_name = orig_name
                    if type_match:
                        extr_type = type_match.group(1).strip()
                    else:
                        extr_type = ""
                        
                    frag_name.append(cleaned_name)
                    frag_type.append(extr_type)
                    #frag_gender.append(gender)
                    frag_brand.append(frag.find_element(by='xpath', value='.//span[@class="brand-name"]').text)
                    frag_price.append(frag.find_element(by='xpath', value='.//div[@class="now-price"]').text)
                    frag_retail_price.append(frag.find_element(by='xpath', value='.//div[@class="was-wrapper"]').text)
                    frag_discount.append(frag.find_element(by='xpath', value='.//span[@class="tag-item discount-label"]').text)
                except Exception as e:
                    print(f"Scraping Error: {e}")
                    continue
            try:
                next_page = driver.find_element(by='xpath', value="//li[contains(@class, 'pagination-next page-item')]//a[contains(@class, 'page-link')]")
                next_page.click()
                page_counter += 1
                time.sleep(2)
            except NoSuchElementException:
                print(f"Last page reached, scraping complete. Last item scraped: {cleaned_name}")
                break

        # Ensuring all cols are of equal length to fit inside the DataFrame.
        max_length = max(len(frag_brand), len(frag_name), len(frag_type), len(frag_price), len(frag_retail_price), len(frag_discount))
        frag_brand.extend([''] * (max_length - len(frag_brand)))
        frag_name.extend([''] * (max_length - len(frag_name)))
        frag_type.extend([''] * (max_length - len(frag_type)))
       #frag_gender.extend([''] * (max_length - len(frag_gender)))
        frag_price.extend([''] * (max_length - len(frag_price)))
        frag_retail_price.extend([''] * (max_length - len(frag_retail_price)))
        frag_discount.extend([''] * (max_length - len(frag_discount)))

        df_frags = pd.DataFrame({
            'BRAND': frag_brand,
            'NAME': frag_name,
            'TYPE': frag_type,
            #'GENDER': frag_gender, 
            'RETAIL_PRICE': frag_retail_price,
            'DISCOUNT': frag_discount,
            'PRICE': frag_price
        })

        df_frags.to_csv('frags.csv', index=False)

        # returning the dataframe in order to access it in the find_notes function.
        return df_frags
        
    df_frags_full = find_frags(driver)

    # Navigates to Parfumo (similar to fragrantica), searches for the fragrance, click the result, and scrape the notes & accords from the page.
    def find_notes(driver, frag_names):
        driver.get("https://www.parfumo.com/Perfumes")
    
        # Cookies modal pops up when loading the page for the first time, clicking the accept button.
        cookies_iframe = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//iframe[contains(@id, 'sp_message_iframe')]")))                                                 
        driver.switch_to.frame(cookies_iframe)
        driver.find_element(By.XPATH, "//button[contains(@class, 'sp_choice_type_11')]").click()
        driver.switch_to.default_content()

        name_pattern_no_conc = regex.compile(r"^(?:(Men's|Ladies|Unisex)\s+)?(?:-\s*)?([^/]+?)(?=\s?Spray\s?|\s*/|\s+by|\s+\(|\s+\d+\.\d+|\s+(?:Aftershave|Bath and Shower Products|Body Spray|Car Diffuser|Cleansers|Deodorant|Diffuser|Eau De Parfum|Eau De Cologne|Eau De Toilette|Extrait de Parfum|Eau de Cologne|Eau de Parfum|Eau de Toilette|Extrait De Parfum|Free Water|Gift Set|Lotion & Oils|Lotions & Creams|Mist|Oil & Serums|Perfume Oil|Room Fragrance|Room Spray|Scented Candle|Scented Cards|Scrubs, Foams & Exfoliants|Shower Gels|Soap|Solid Parfum|Tools|Wash|EDP|EDT|\bIntense\b)|$)|\((m|u|w)\)")
                                     
        # Some fragrances start with the same name, but have a different perfume concentration (EDP, EDT, Cologne, etc)
        results_already_scraped = set()
        
        fragrance_types = ["edt", "edp", "parfum", "intense"]
        
        for cleaned_name in frag_names:
            try:
                name_match_no_conc = regex.match(name_pattern_no_conc, cleaned_name)
                if name_match_no_conc:
                    name_no_conc = name_match_no_conc.group(2)   
                else:
                    name_no_conc = cleaned_name
                frag_searchbox = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//input[contains(@class, 'action-livesearch')]")))
                frag_searchbox.clear()
                time.sleep(0.5)
                frag_searchbox.send_keys(cleaned_name)
                search_results_list = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, "//table[contains(@id, 'ls-perfumes')]//tr")))
                
                for result in search_results_list:
                    try:
                        name_from_result = result.find_element(By.XPATH, ".//div[contains(@class, 'name bold')]").text
                        type_tag = ""
                        try:
                            type_tag = result.find_element(By.XPATH, ".//span[contains(@class, 'upper')]").text
                        except NoSuchElementException:
                            pass
                    
                        # Checking if this result has already been scraped.
                        try:
                            if (name_from_result, name_match_no_conc, type_tag) in results_already_scraped:
                                for fragrance in fragrance_types:
                                    if fragrance.lower() in cleaned_name.lower():  # Case-insensitive check
                                        print(f"Found {fragrance} in {cleaned_name}")
                                        break
                            else:
                                # Re-locate the clickable element to avoid stale element references.
                                WebDriverWait(driver, 5).until(EC.element_to_be_clickable(result)).click()

                                # Need to grab the notes / accords here.
                                
                                results_already_scraped.add((name_from_result, name_match_no_conc, type_tag))
                                break
                        except StaleElementReferenceException as e:
                            print(f"Error processing the result for {name_from_result}: {e}")
                    except Exception as e:
                        print(f"Error processing the result for {cleaned_name}: {e}")
                continue
            except TimeoutException:
                print(f"No search results returned for {cleaned_name}.")
                
        print("Results:-------------------------")
        for (name_from_result, name_no_conc, type_tag) in results_already_scraped:
            print(f"Name from Result: {name_from_result.text}\nName No Conc.: {name_no_conc.text}\nType Tag: {type_tag}\n")
        
    # find_notes(driver, df_frags_full["NAME"])
    
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

Scraping Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="was-wrapper"]"}
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF751849C65+28805]
	(No symbol) [0x00007FF7517AF6F0]
	(No symbol) [0x00007FF75164578A]
	(No symbol) [0x00007FF7516990BE]
	(No symbol) [0x00007FF7516993AC]
	(No symbol) [0x00007FF75168C50C]
	(No symbol) [0x00007FF7516BF20F]
	(No symbol) [0x00007FF75168C3D6]
	(No symbol) [0x00007FF7516BF3E0]
	(No symbol) [0x00007FF7516DF3C4]
	(No symbol) [0x00007FF7516BEF73]
	(No symbol) [0x00007FF75168A758]
	(No symbol) [0x00007FF75168B8C1]
	GetHandleVerifier [0x00007FF751B75B9D+3354557]
	GetHandleVerifier [0x00007FF751B88460+3430528]
	GetHandleVerifier [0x00007FF751B7E66D+3390093]
	GetHandleVerifier [0x00007FF75191080B+842795]
	(No symbol) [0x00007FF7517BAACF]
	(No symbol) [0x00007FF7517B66E4]
	(No symbol) [0x00007FF7517B687D]
	(No symbol) [0x00007FF7517A5D69]
	BaseThreadInitThunk [0x00007FFCFFFF25